# Import Library

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import json

# Creating DataBase functions
import sqlite3
from sqlite3 import Error

import time
import matplotlib.pyplot as plt

from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set up Token


In [2]:
client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Get Dataset

In [4]:
#the dataset is too big to upload on github so there will be no corresponding folder
df = pd.read_csv('../../spotify_playlists/spotify_dataset.csv',error_bad_lines=False,warn_bad_lines=False)
# Rename columns
df.rename(index=str, 
          columns={
              ' "artistname"': "artist",
              ' "trackname"': "trackname",
              ' "playlistname"': "playlistname"},
          inplace=True)


In [ ]:
#Create User-Playlistname Column
user_df['user_playlistname'] = user_df['user_id'] + '--' + user_df['playlistname']

In [6]:
#get playlist's size
user_df['playlist_size'] = user_df.groupby('user_playlistname')['user_playlistname'].transform('size')

In [ ]:
#get playlist's position
user_df['playlist_position'] = user_df.groupby('user_playlistname').cumcount(ascending=True)

In [ ]:
# Only select playlists that have more than 10 and less than 50 songs
user_df = user_df[user_df.playlist_size >= 10]
user_df = user_df[user_df.playlist_size <= 50]
user_df['user_playlistname'].nunique()

In [7]:
#calculate the sum of null value in each row
user_df['playlist_null_count'] = user_df.groupby('user_playlistname')['artist'].transform(lambda group: group.isnull().sum())
#drop NAs
user_df = user_df[user_df['playlist_null_count'] == 0].dropna()

In [ ]:
#build search string
user_df['search_string'] = user_df['artist'] + ' ' + user_df['trackname']
user_df['search_string'].nunique()

In [ ]:
all_tracks = user_df[['search_string']].drop_duplicates().reset_index(drop=True)
all_tracks.to_csv('../../user_playlists.csv')

# Creating database functions

## Write some functions to create database table and insert values


In [5]:
#create a database connection
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as err:
        print(err)
 
    return conn

# Creating table function
def create_table(conn, query):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param query: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(query)
    except Error as err:
        print('ERROR BY CREATE TABLE', err)

# function to insert values        
def insert_values(conn, query, values):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param query: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(query, values)
    except Error as e:
        pass

## Create database for all tracks' ids

In [7]:
def get_track_id(search_string):
    try:
        return sp.search(search_string,limit=1)['tracks']['items'][0]['id']
    except:
        return ''

#this function creates database and inserts values. 
def get_playlist_id_to_db(offset):
    #create db
    db = 'Spotify.db'
    create_table_query = """
                    CREATE TABLE IF NOT EXISTS all_song_ids (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        search_string VARCHAR(255),
                        track_id VARCHAR(100)
                    );
                    """
    ## sql command for inserting values
    insert_value_query = 'INSERT INTO all_song_ids (search_string, track_id) VALUES (?,?);'

    start = 0
    print(f'{offset*10000}-{offset*10000 + 10000}')
    search_strings = all_tracks['search_string'][offset*10000:offset*10000 + 10000].tolist()
    results = []
    
    # just going through the spotify object and retriving data.     
    for i, search_string in tqdm(enumerate(search_strings)):
        if i % 500 == 0:
            print(i)
        results.append(get_track_id(search_string))
    print('Search string - ', len(results))
    # creating a connection with database
    conn = create_connection(db)
    create_table(conn, create_table_query)
    print('Create table')
    # inserting values from the list
    for search_string, result in zip(search_strings, results):
        values = (search_string, result)
        insert_values(conn, insert_value_query, values)
        conn.commit()
    conn.close()

In [8]:
all_tracks = pd.read_csv('../all_tracks.csv')

In [9]:
all_tracks.head()

,search_string
0,Noah And The Whale 5 Years Time
1,Noel Gallagher's High Flying Birds AKA... What...
2,Pearl Jam Alive
3,Tom Petty And The Heartbreakers American Girl
4,Bruce Springsteen Atlantic City


In [10]:
%%time
## sql_command for creating table. 
create_table_query = """
                    CREATE TABLE IF NOT EXISTS all_song_ids (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        search_string VARCHAR(255),
                        track_id VARCHAR(100)
                    );
                    """

## sql command for inserting values
insert_value_query = 'INSERT INTO all_song_ids (search_string, track_id) VALUES (?,?);'

## this function creates database and inserts values. 
# start from 40
for i in range(44, 50):
    get_playlist_id_to_db(i)  # offset = 50000

440000-450000


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500

Search string -  10000
Create table
450000-460000


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500

Search string -  10000
Create table
460000-470000


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500

Search string -  10000
Create table
470000-480000


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500

Search string -  10000
Create table
480000-490000


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500

Search string -  10000
Create table
490000-500000


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500

Search string -  10000
Create table
Wall time: 1h 59min 15s


In [10]:
# %%time
# from concurrent.futures import as_completed, ProcessPoolExecutor

# offsets = [i for i in range(40,44)]
# with ProcessPoolExecutor(max_workers=4) as executor:
#     executor.map(get_playlist_id_to_db, offsets)

Wall time: 884 ms


In [ ]:
#../../track_ids_csv/track_id-0-5000.csv

In [444]:
# final_df = pd.concat([final_df,track_df]).reset_index(drop=True)
final_df

,search_string,track_id
0,Noah And The Whale 5 Years Time,1KlDcHrlDPUJJPmMup3tUp
1,Noel Gallagher's High Flying Birds AKA... What...,5mljJfOTZR3nF4ExmRLTjv
2,Pearl Jam Alive,1L94M3KIu7QluZe63g64rv
3,Tom Petty And The Heartbreakers American Girl,7MRyJPksH3G2cXHN8UKYzP
4,Bruce Springsteen Atlantic City,1Vp8U39YNsDfd6yVuaUq12
...,...,...
655095,Aly Tejas Nathalie - Adage 6/8,0KyTcSml8gfHIsO2OOQMmj
655096,Aly Tejas Pirouettes 4/4 - Bonus Track,7KcaSZjYaEeLLqO1ccc6rW
655097,Aly Tejas Port De Bras 3/4,6S0pEd8Ob5TsHFR8ZkSFJ5
655098,Aly Tejas Rond De Jambe En L’air 3/4,0dN2zVsBrsvUdPvV5sV6sg


In [445]:
final_df.to_csv('../../track_ids_csv/track_id.csv')

In [447]:
non_duplicate_track_id = final_df.drop_duplicates()
non_duplicate_track_id

,search_string,track_id
0,Noah And The Whale 5 Years Time,1KlDcHrlDPUJJPmMup3tUp
1,Noel Gallagher's High Flying Birds AKA... What...,5mljJfOTZR3nF4ExmRLTjv
2,Pearl Jam Alive,1L94M3KIu7QluZe63g64rv
3,Tom Petty And The Heartbreakers American Girl,7MRyJPksH3G2cXHN8UKYzP
4,Bruce Springsteen Atlantic City,1Vp8U39YNsDfd6yVuaUq12
...,...,...
655095,Aly Tejas Nathalie - Adage 6/8,0KyTcSml8gfHIsO2OOQMmj
655096,Aly Tejas Pirouettes 4/4 - Bonus Track,7KcaSZjYaEeLLqO1ccc6rW
655097,Aly Tejas Port De Bras 3/4,6S0pEd8Ob5TsHFR8ZkSFJ5
655098,Aly Tejas Rond De Jambe En L’air 3/4,0dN2zVsBrsvUdPvV5sV6sg


In [448]:
non_duplicate_track_id.to_csv('../../track_ids_csv/non_duplicate_track_id.csv')

## Create database for all tracks' metadata based on the track's id

In [6]:
#testing
non_duplicate_track_id = pd.read_csv('../../track_ids_csv/non_duplicate_track_id.csv')
non_duplicate_track_id.head(15)

,Unnamed: 0,search_string,track_id
0,0,Noah And The Whale 5 Years Time,1KlDcHrlDPUJJPmMup3tUp
1,1,Noel Gallagher's High Flying Birds AKA... What...,5mljJfOTZR3nF4ExmRLTjv
2,2,Pearl Jam Alive,1L94M3KIu7QluZe63g64rv
3,3,Tom Petty And The Heartbreakers American Girl,7MRyJPksH3G2cXHN8UKYzP
4,4,Bruce Springsteen Atlantic City,1Vp8U39YNsDfd6yVuaUq12
5,5,Madness Baggy Trousers,3Sbnhrpec0ULbeWBsA6GS6
6,6,Pearl Jam Black,5Xak5fmy089t0FYmh3VJiY
7,7,Bruce Springsteen Born to Run,6hTcuIQa0sxrrByu9wTD7s
8,8,Spector Chevy Thunder,6ikzdvfsPbdaGpXmeEeaK2
9,9,Miles Kane Come Closer,20uVZbYFKshjrdDHcVy9w9


In [480]:
test_res = []
for i in range(0,5):
    test_res.append(sp.audio_features(all_tracks.loc[i,'track_id'])[0])
test_res= list(filter(None, test_res))
f_df = pd.DataFrame(test_res)
f_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.803,0.548,0,-7.103,1,0.1200,0.351000,0.000000,0.0953,0.750,121.942,audio_features,1KlDcHrlDPUJJPmMup3tUp,spotify:track:1KlDcHrlDPUJJPmMup3tUp,https://api.spotify.com/v1/tracks/1KlDcHrlDPUJ...,https://api.spotify.com/v1/audio-analysis/1KlD...,214920,4
1,0.447,0.927,6,-3.300,0,0.0508,0.000297,0.362000,0.0972,0.104,119.034,audio_features,5mljJfOTZR3nF4ExmRLTjv,spotify:track:5mljJfOTZR3nF4ExmRLTjv,https://api.spotify.com/v1/tracks/5mljJfOTZR3n...,https://api.spotify.com/v1/audio-analysis/5mlj...,264480,4
2,0.284,0.875,11,-6.069,0,0.0422,0.007520,0.000461,0.4020,0.595,75.009,audio_features,1L94M3KIu7QluZe63g64rv,spotify:track:1L94M3KIu7QluZe63g64rv,https://api.spotify.com/v1/tracks/1L94M3KIu7Ql...,https://api.spotify.com/v1/audio-analysis/1L94...,340907,4
3,0.550,0.824,2,-5.988,1,0.0334,0.448000,0.000127,0.3660,0.777,114.512,audio_features,7MRyJPksH3G2cXHN8UKYzP,spotify:track:7MRyJPksH3G2cXHN8UKYzP,https://api.spotify.com/v1/tracks/7MRyJPksH3G2...,https://api.spotify.com/v1/audio-analysis/7MRy...,214733,4
4,0.475,0.429,4,-10.577,1,0.0367,0.311000,0.000000,0.3760,0.340,108.034,audio_features,1Vp8U39YNsDfd6yVuaUq12,spotify:track:1Vp8U39YNsDfd6yVuaUq12,https://api.spotify.com/v1/tracks/1Vp8U39YNsDf...,https://api.spotify.com/v1/audio-analysis/1Vp8...,236653,4


In [474]:
for i in f[0].values():
    print(i)

0.803
0.548
0
-7.103
1
0.12
0.351
0
0.0953
0.75
121.942
audio_features
1KlDcHrlDPUJJPmMup3tUp
spotify:track:1KlDcHrlDPUJJPmMup3tUp
https://api.spotify.com/v1/tracks/1KlDcHrlDPUJJPmMup3tUp
https://api.spotify.com/v1/audio-analysis/1KlDcHrlDPUJJPmMup3tUp
214920
4


In [498]:
for i, res in enumerate(test_res):
    values = tuple(res.values())
    print(i, values)
    print('\n')

0 (0.803, 0.548, 0, -7.103, 1, 0.12, 0.351, 0, 0.0953, 0.75, 121.942, 'audio_features', '1KlDcHrlDPUJJPmMup3tUp', 'spotify:track:1KlDcHrlDPUJJPmMup3tUp', 'https://api.spotify.com/v1/tracks/1KlDcHrlDPUJJPmMup3tUp', 'https://api.spotify.com/v1/audio-analysis/1KlDcHrlDPUJJPmMup3tUp', 214920, 4)


1 (0.447, 0.927, 6, -3.3, 0, 0.0508, 0.000297, 0.362, 0.0972, 0.104, 119.034, 'audio_features', '5mljJfOTZR3nF4ExmRLTjv', 'spotify:track:5mljJfOTZR3nF4ExmRLTjv', 'https://api.spotify.com/v1/tracks/5mljJfOTZR3nF4ExmRLTjv', 'https://api.spotify.com/v1/audio-analysis/5mljJfOTZR3nF4ExmRLTjv', 264480, 4)


2 (0.284, 0.875, 11, -6.069, 0, 0.0422, 0.00752, 0.000461, 0.402, 0.595, 75.009, 'audio_features', '1L94M3KIu7QluZe63g64rv', 'spotify:track:1L94M3KIu7QluZe63g64rv', 'https://api.spotify.com/v1/tracks/1L94M3KIu7QluZe63g64rv', 'https://api.spotify.com/v1/audio-analysis/1L94M3KIu7QluZe63g64rv', 340907, 4)


3 (0.55, 0.824, 2, -5.988, 1, 0.0334, 0.448, 0.000127, 0.366, 0.777, 114.512, 'audio_features', 

In [463]:
sp.audio_features(all_tracks.loc[0,'track_id'])[0].keys()

dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])

In [11]:
all_tracks = pd.read_csv('../../all_song_ids.csv')
all_tracks.shape

(951163, 3)

In [22]:
def get_song_features(id_,list_=None):
    '''
    Get song features for each song in a list
    '''
    try:
        list_.append(sp.audio_features(id_)[0])
    except:
        list_.append(id_)

def get_track_features_to_db(offset):
    #create db
    db = 'Spotify.db'
    create_table_query = """
                    CREATE TABLE IF NOT EXISTS song_features (
                        rowid INTEGER PRIMARY KEY AUTOINCREMENT,
                        danceability REAL,
                        energy REAL,
                        key INTEGER,
                        loudness REAL,
                        mode INTEGER,
                        speechiness REAL,
                        acousticness REAL,
                        instrumentalness REAL,
                        liveness REAL,
                        valence REAL,
                        tempo REAL,
                        type TEXT,
                        id TEXT,
                        uri TEXT,
                        track_href TEXT,
                        analysis_url TEXT,
                        duration_ms INTEGER,
                        time_signature INTEGER
                    );
                    """
    ## sql command for inserting values
    insert_value_query = 'INSERT INTO song_features (danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, type, id, uri, track_href, analysis_url, duration_ms, time_signature) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'

    # print(f'{offset*1000}-{offset*1000+1000}')
    # track_ids = all_tracks['track_id'][offset*1000:offset*1000+1000].tolist()

    track_ids = all_tracks['track_id'][951000:].tolist()
    results = []
    
    
    # just going through the spotify object and retriving data.     
    for i, track_id in tqdm(enumerate(track_ids)):
        if i % 50 == 0:
            print(i)
        try: 
            get_song_features(track_id, results)
        except Exception as err:
            print(i, err)
    
    print('track id - ', len(results))

    #creating a connection with database
    conn = create_connection(db)
    create_table(conn, create_table_query)
    print('Create table')
    err = 0
    # inserting values from the list
    for result in results:
        try:
            values = tuple(result.values())
            insert_values(conn, insert_value_query, values)
            conn.commit()
        except:
            err += 1
    
    conn.close()
    print('Finish insert values to the table - ', err)

In [23]:
# for offset in range(0):
get_track_features_to_db(0)

0
50
100
150

track id -  163
Create table
Finish insert values to the table -  46
